In [ ]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# EfficientNet (pip install efficientnet-pytorch)
from efficientnet_pytorch import EfficientNet
from tqdm import tqdm  # 학습 진행률 표시

# ==========================
# ✅ SEED 설정 (재현성 확보)
# ==========================
seed = 2021
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ==============================
# ✅ Early Stopping 클래스
# ==============================
class EarlyStopping:
    def __init__(self, patience=10, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# ==============================
# ✅ Residual Block 정의
# ==============================
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if in_channels != out_channels or stride != 1:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = self.shortcut(x)
        x = nn.ReLU()(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        x += residual
        return nn.ReLU()(x)

# ==============================
# ✅ ASPP Block 정의
# ==============================
class ASPP(nn.Module):
    def __init__(self, in_channels, out_channels=256):
        super(ASPP, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.conv3_1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, dilation=1)
        self.conv3_6 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=6, dilation=6)
        self.conv3_12 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=12, dilation=12)
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.conv1_for_global = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.conv1x1_out = nn.Conv2d(out_channels * 5, out_channels, kernel_size=1)

    def forward(self, x):
        conv1_out = self.conv1(x)
        conv3_1_out = self.conv3_1(x)
        conv3_6_out = self.conv3_6(x)
        conv3_12_out = self.conv3_12(x)

        global_avg = self.global_avg_pool(x)
        global_avg = self.conv1_for_global(global_avg)
        global_avg = nn.functional.interpolate(global_avg, size=conv1_out.shape[2:], mode='bilinear', align_corners=True)

        out = torch.cat([conv1_out, conv3_1_out, conv3_6_out, conv3_12_out, global_avg], dim=1)
        return self.conv1x1_out(out)

# ==============================
# ✅ EfficientNet + VGG + Residual + ASPP 모델 정의
# ==============================
class HybridModel(nn.Module):
    def __init__(self, num_classes=6):
        super(HybridModel, self).__init__()

        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b0')
        self.efficientnet._fc = nn.Identity()

        self.vgg = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
        self.vgg.classifier = nn.Identity()

        efficientnet_out_features = self.efficientnet._conv_head.out_channels  # 1280
        vgg_out_features = 512  

        self.efficientnet_pool = nn.AdaptiveAvgPool2d((7, 7))
        self.vgg_pool = nn.AdaptiveAvgPool2d((7, 7))

        self.eff_conv1x1 = nn.Conv2d(efficientnet_out_features, 1024, kernel_size=1)
        self.vgg_conv1x1 = nn.Conv2d(vgg_out_features, 1024, kernel_size=1)

        self.res_block = ResidualBlock(2048, 2048)  # Residual Block 추가

        self.aspp = ASPP(in_channels=2048, out_channels=256)

        self.classifier = nn.Sequential(
            nn.Linear(256 * 7 * 7, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        eff_features = self.efficientnet.extract_features(x)
        eff_features = self.efficientnet_pool(eff_features)
        eff_features = self.eff_conv1x1(eff_features)

        vgg_features = self.vgg.features(x)
        vgg_features = self.vgg_pool(vgg_features)
        vgg_features = self.vgg_conv1x1(vgg_features)

        fused_features = torch.cat([eff_features, vgg_features], dim=1)  

        fused_features = self.res_block(fused_features)  # Residual Block 적용
        aspp_features = self.aspp(fused_features)

        aspp_features = aspp_features.view(aspp_features.size(0), -1)
        return self.classifier(aspp_features)

# ==============================
# ✅ 데이터 로딩
# ==============================
image_size = 224
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset_path = "C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel"

dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# ==============================
# ✅ 모델, 손실 함수, 최적화기 설정
# ==============================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HybridModel(num_classes=6).to(device)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# ✅ Early Stopping 적용
early_stopping = EarlyStopping(patience=10)

# ✅ 학습 함수
def train(model, train_loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 평가 함수
def evaluate(model, val_loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 학습 루프
best_acc = 0.0
num_epochs = 100

for epoch in range(num_epochs):
    print(f"\n[Epoch {epoch+1}/{num_epochs}]")
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    val_loss, val_acc = evaluate(model, test_loader, criterion)

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")
        print("  [*] Best model saved.")

    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("Early Stopping Triggered!")
        break

print(f"\nBest Accuracy: {best_acc:.4f}")


In [3]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from efficientnet_pytorch import EfficientNet
from tqdm import tqdm

# ✅ SEED 설정 (재현성 확보)
seed = 2021
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ✅ EarlyStopping
class EarlyStopping:
    def __init__(self, patience=10, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False
    def __call__(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# ✅ Residual Block 정의
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if in_channels != out_channels or stride != 1:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = self.shortcut(x)
        x = nn.ReLU()(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        x += residual
        return nn.ReLU()(x)

# ✅ ASPP Block 정의
class ASPP(nn.Module):
    def __init__(self, in_channels, out_channels=256):
        super(ASPP, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.conv3_1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, dilation=1)
        self.conv3_6 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=6, dilation=6)
        self.conv3_12 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=12, dilation=12)
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.conv1_for_global = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.conv1x1_out = nn.Conv2d(out_channels * 5, out_channels, kernel_size=1)

    def forward(self, x):
        conv1_out = self.conv1(x)
        conv3_1_out = self.conv3_1(x)
        conv3_6_out = self.conv3_6(x)
        conv3_12_out = self.conv3_12(x)

        global_avg = self.global_avg_pool(x)
        global_avg = self.conv1_for_global(global_avg)
        global_avg = nn.functional.interpolate(global_avg, size=conv1_out.shape[2:], mode='bilinear', align_corners=True)

        out = torch.cat([conv1_out, conv3_1_out, conv3_6_out, conv3_12_out, global_avg], dim=1)
        return self.conv1x1_out(out)

# ✅ Hybrid Model (EfficientNet + VGG + Residual + ASPP)
class HybridModel(nn.Module):
    def __init__(self, num_classes=6):
        super(HybridModel, self).__init__()
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b0')
        self.efficientnet._fc = nn.Identity()

        self.vgg = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
        self.vgg.classifier = nn.Identity()

        efficientnet_out_features = self.efficientnet._conv_head.out_channels  # 1280
        vgg_out_features = 512  

        self.efficientnet_pool = nn.AdaptiveAvgPool2d((7, 7))
        self.vgg_pool = nn.AdaptiveAvgPool2d((7, 7))

        self.eff_conv1x1 = nn.Conv2d(efficientnet_out_features, 1024, kernel_size=1)
        self.vgg_conv1x1 = nn.Conv2d(vgg_out_features, 1024, kernel_size=1)

        self.res_block = ResidualBlock(2048, 2048)  
        self.aspp = ASPP(in_channels=2048, out_channels=256)

        self.classifier = nn.Sequential(
            nn.Linear(256 * 7 * 7, 512),
            nn.ReLU(),
            nn.Dropout(0.5),  
            nn.Linear(512, num_classes)
        )

        # ✅ EfficientNet의 일부 블록만 Unfreeze (초기 블록 2개만 학습)
        for name, param in self.efficientnet.named_parameters():
            if "blocks.0" in name or "blocks.1" in name:  # 블록 0, 1만 학습
                param.requires_grad = True
            else:
                param.requires_grad = False

    def forward(self, x):
        eff_features = self.efficientnet.extract_features(x)
        eff_features = self.efficientnet_pool(eff_features)
        eff_features = self.eff_conv1x1(eff_features)

        vgg_features = self.vgg.features(x)
        vgg_features = self.vgg_pool(vgg_features)
        vgg_features = self.vgg_conv1x1(vgg_features)

        fused_features = torch.cat([eff_features, vgg_features], dim=1)  
        fused_features = self.res_block(fused_features)  
        aspp_features = self.aspp(fused_features)

        aspp_features = aspp_features.view(aspp_features.size(0), -1)
        return self.classifier(aspp_features)

# ==============================
# ✅ 데이터 로딩
# ==============================
image_size = 224
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset_path = "C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel"

dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# ==============================
# ✅ 모델, 손실 함수, 최적화기 설정
# ==============================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HybridModel(num_classes=6).to(device)  # 모델을 먼저 생성해야 함!

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# ✅ Early Stopping 적용
early_stopping = EarlyStopping(patience=10)

# ✅ 학습 함수
def train(model, train_loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 평가 함수
def evaluate(model, val_loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 학습 루프
best_acc = 0.0
num_epochs = 100

for epoch in range(num_epochs):
    print(f"\n[Epoch {epoch+1}/{num_epochs}]")
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    val_loss, val_acc = evaluate(model, test_loader, criterion)

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")
        print("  [*] Best model saved.")

    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("Early Stopping Triggered!")
        break

print(f"\nBest Accuracy: {best_acc:.4f}")



Loaded pretrained weights for efficientnet-b0

[Epoch 1/100]


Train Loss: 10.3049 | Train Acc: 0.1540 | Val Loss: 9.5322 | Val Acc: 0.1808
  [*] Best model saved.

[Epoch 2/100]


Train Loss: 2.0264 | Train Acc: 0.2274 | Val Loss: 1.7281 | Val Acc: 0.2768
  [*] Best model saved.

[Epoch 3/100]


Train Loss: 1.8039 | Train Acc: 0.2401 | Val Loss: 1.7300 | Val Acc: 0.2147

[Epoch 4/100]


Train Loss: 1.7511 | Train Acc: 0.2740 | Val Loss: 1.6680 | Val Acc: 0.2203

[Epoch 5/100]


...

Train Loss: 0.5835 | Train Acc: 0.9760 | Val Loss: 0.9326 | Val Acc: 0.7966

[Epoch 22/100]


Train Loss: 0.5823 | Train Acc: 0.9760 | Val Loss: 0.8043 | Val Acc: 0.8983
  [*] Best model saved.

[Epoch 23/100]


Train Loss: 0.6068 | Train Acc: 0.9703 | Val Loss: 0.8458 | Val Acc: 0.8362

[Epoch 24/100]


Train Loss: 0.5818 | Train Acc: 0.9718 | Val Loss: 0.8271 | Val Acc: 0.8531

[Epoch 25/100]


Train Loss: 0.5706 | Train Acc: 0.9816 | Val Loss: 0.9914 | Val Acc: 0.7627
Early Stopping Triggered!

Best Accuracy: 0.8983


In [4]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from efficientnet_pytorch import EfficientNet
from tqdm import tqdm

# ✅ SEED 설정 (재현성 확보)
seed = 2021
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ✅ EarlyStopping
class EarlyStopping:
    def __init__(self, patience=10, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False
    def __call__(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# ✅ Residual Block 정의
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if in_channels != out_channels or stride != 1:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = self.shortcut(x)
        x = nn.ReLU()(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        x += residual
        return nn.ReLU()(x)

# ✅ ASPP Block 정의
class ASPP(nn.Module):
    def __init__(self, in_channels, out_channels=256):
        super(ASPP, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.conv3_1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, dilation=1)
        self.conv3_6 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=6, dilation=6)
        self.conv3_12 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=12, dilation=12)
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.conv1_for_global = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.conv1x1_out = nn.Conv2d(out_channels * 5, out_channels, kernel_size=1)

    def forward(self, x):
        conv1_out = self.conv1(x)
        conv3_1_out = self.conv3_1(x)
        conv3_6_out = self.conv3_6(x)
        conv3_12_out = self.conv3_12(x)

        global_avg = self.global_avg_pool(x)
        global_avg = self.conv1_for_global(global_avg)
        global_avg = nn.functional.interpolate(global_avg, size=conv1_out.shape[2:], mode='bilinear', align_corners=True)

        out = torch.cat([conv1_out, conv3_1_out, conv3_6_out, conv3_12_out, global_avg], dim=1)
        return self.conv1x1_out(out)

# ✅ Hybrid Model (EfficientNet + VGG + Residual + ASPP)
class HybridModel(nn.Module):
    def __init__(self, num_classes=6):
        super(HybridModel, self).__init__()
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b0')
        self.efficientnet._fc = nn.Identity()

        self.vgg = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
        self.vgg.classifier = nn.Identity()

        efficientnet_out_features = self.efficientnet._conv_head.out_channels  # 1280
        vgg_out_features = 512  

        self.efficientnet_pool = nn.AdaptiveAvgPool2d((7, 7))
        self.vgg_pool = nn.AdaptiveAvgPool2d((7, 7))

        self.eff_conv1x1 = nn.Conv2d(efficientnet_out_features, 1024, kernel_size=1)
        self.vgg_conv1x1 = nn.Conv2d(vgg_out_features, 1024, kernel_size=1)

        self.res_block = ResidualBlock(2048, 2048)  
        self.aspp = ASPP(in_channels=2048, out_channels=256)

        self.classifier = nn.Sequential(
            nn.Linear(256 * 7 * 7, 512),
            nn.ReLU(),
            nn.Dropout(0.3),  
            nn.Linear(512, num_classes)
        )

        # ✅ EfficientNet의 일부 블록만 Unfreeze (초기 블록 2개만 학습)
        for name, param in self.efficientnet.named_parameters():
            if "blocks.0" in name or "blocks.1" in name:  # 블록 0, 1만 학습
                param.requires_grad = True
            else:
                param.requires_grad = False

    def forward(self, x):
        eff_features = self.efficientnet.extract_features(x)
        eff_features = self.efficientnet_pool(eff_features)
        eff_features = self.eff_conv1x1(eff_features)

        vgg_features = self.vgg.features(x)
        vgg_features = self.vgg_pool(vgg_features)
        vgg_features = self.vgg_conv1x1(vgg_features)

        fused_features = torch.cat([eff_features, vgg_features], dim=1)  
        fused_features = self.res_block(fused_features)  
        aspp_features = self.aspp(fused_features)

        aspp_features = aspp_features.view(aspp_features.size(0), -1)
        return self.classifier(aspp_features)

# ==============================
# ✅ 데이터 로딩
# ==============================
image_size = 224
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset_path = "C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel"

dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# ==============================
# ✅ 모델, 손실 함수, 최적화기 설정
# ==============================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HybridModel(num_classes=6).to(device)  # 모델을 먼저 생성해야 함!

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# ✅ Early Stopping 적용
early_stopping = EarlyStopping(patience=10)

# ✅ 학습 함수
def train(model, train_loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 평가 함수
def evaluate(model, val_loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 학습 루프
best_acc = 0.0
num_epochs = 100

for epoch in range(num_epochs):
    print(f"\n[Epoch {epoch+1}/{num_epochs}]")
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    val_loss, val_acc = evaluate(model, test_loader, criterion)

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")
        print("  [*] Best model saved.")

    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("Early Stopping Triggered!")
        break

print(f"\nBest Accuracy: {best_acc:.4f}")



Loaded pretrained weights for efficientnet-b0

[Epoch 1/100]


Train Loss: 9.2672 | Train Acc: 0.1667 | Val Loss: 3.3439 | Val Acc: 0.2994
  [*] Best model saved.

[Epoch 2/100]


Train Loss: 1.8356 | Train Acc: 0.2542 | Val Loss: 1.6732 | Val Acc: 0.3842
  [*] Best model saved.

[Epoch 3/100]


Train Loss: 1.6044 | Train Acc: 0.3842 | Val Loss: 1.4448 | Val Acc: 0.5593
  [*] Best model saved.

[Epoch 4/100]


Train Loss: 1.3220 | Train Acc: 0.5636 | Val Loss: 1.1580 | Val Acc: 0.6893
  [*] Best model saved.

[Epoch 5/100]


...

Train Loss: 0.5116 | Train Acc: 0.9915 | Val Loss: 0.8478 | Val Acc: 0.8588

[Epoch 29/100]


Train Loss: 0.5063 | Train Acc: 0.9915 | Val Loss: 0.8160 | Val Acc: 0.8588

[Epoch 30/100]


Train Loss: 0.5027 | Train Acc: 0.9887 | Val Loss: 0.8381 | Val Acc: 0.8362

[Epoch 31/100]


Train Loss: 0.5101 | Train Acc: 0.9873 | Val Loss: 0.7589 | Val Acc: 0.8757

[Epoch 32/100]


Train Loss: 0.4992 | Train Acc: 0.9958 | Val Loss: 0.7722 | Val Acc: 0.8927
Early Stopping Triggered!

Best Accuracy: 0.8927


In [1]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from efficientnet_pytorch import EfficientNet
from tqdm import tqdm

# ✅ SEED 설정 (재현성 확보)
seed = 2021
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ✅ EarlyStopping
class EarlyStopping:
    def __init__(self, patience=10, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False
    def __call__(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# ✅ Residual Block 정의
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if in_channels != out_channels or stride != 1:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = self.shortcut(x)
        x = nn.ReLU()(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        x += residual
        return nn.ReLU()(x)

# ✅ ASPP Block 정의
class ASPP(nn.Module):
    def __init__(self, in_channels, out_channels=256):
        super(ASPP, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.conv3_1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, dilation=1)
        self.conv3_6 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=6, dilation=6)
        self.conv3_12 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=12, dilation=12)
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.conv1_for_global = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.conv1x1_out = nn.Conv2d(out_channels * 5, out_channels, kernel_size=1)

    def forward(self, x):
        conv1_out = self.conv1(x)
        conv3_1_out = self.conv3_1(x)
        conv3_6_out = self.conv3_6(x)
        conv3_12_out = self.conv3_12(x)

        global_avg = self.global_avg_pool(x)
        global_avg = self.conv1_for_global(global_avg)
        global_avg = nn.functional.interpolate(global_avg, size=conv1_out.shape[2:], mode='bilinear', align_corners=True)

        out = torch.cat([conv1_out, conv3_1_out, conv3_6_out, conv3_12_out, global_avg], dim=1)
        return self.conv1x1_out(out)

# ✅ Hybrid Model (EfficientNet + VGG + Residual + ASPP)
class HybridModel(nn.Module):
    def __init__(self, num_classes=6):
        super(HybridModel, self).__init__()
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b0')
        self.efficientnet._fc = nn.Identity()

        self.vgg = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
        self.vgg.classifier = nn.Identity()

        efficientnet_out_features = self.efficientnet._conv_head.out_channels  # 1280
        vgg_out_features = 512  

        self.efficientnet_pool = nn.AdaptiveAvgPool2d((7, 7))
        self.vgg_pool = nn.AdaptiveAvgPool2d((7, 7))

        self.eff_conv1x1 = nn.Conv2d(efficientnet_out_features, 1024, kernel_size=1)
        self.vgg_conv1x1 = nn.Conv2d(vgg_out_features, 1024, kernel_size=1)

        self.res_block = ResidualBlock(2048, 2048)  
        self.aspp = ASPP(in_channels=2048, out_channels=256)

        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))  # GAP 추가
        self.classifier = nn.Sequential(
            nn.Linear(256, 512),  # 256 차원에서 512 차원으로 축소
            nn.ReLU(),
            nn.Dropout(0.3),  
            nn.Linear(512, num_classes)
        )


        # ✅ EfficientNet의 일부 블록만 Unfreeze (초기 블록 2개만 학습)
        for name, param in self.efficientnet.named_parameters():
            if "blocks.0" in name or "blocks.1" in name:  # 블록 0, 1만 학습
                param.requires_grad = True
            else:
                param.requires_grad = False

    def forward(self, x):
        eff_features = self.efficientnet.extract_features(x)
        eff_features = self.efficientnet_pool(eff_features)
        eff_features = self.eff_conv1x1(eff_features)

        vgg_features = self.vgg.features(x)
        vgg_features = self.vgg_pool(vgg_features)
        vgg_features = self.vgg_conv1x1(vgg_features)

        fused_features = torch.cat([eff_features, vgg_features], dim=1)  
        fused_features = self.res_block(fused_features)  
        aspp_features = self.aspp(fused_features)
        gap_features = self.global_avg_pool(aspp_features)
        flattened_features = gap_features.view(gap_features.size(0), -1)
        return self.classifier(flattened_features)


# ==============================
# ✅ 데이터 로딩
# ==============================
image_size = 224
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset_path = "C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel"

dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# ==============================
# ✅ 모델, 손실 함수, 최적화기 설정
# ==============================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HybridModel(num_classes=6).to(device)  # 모델을 먼저 생성해야 함!

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# ✅ Early Stopping 적용
early_stopping = EarlyStopping(patience=10)

# ✅ 학습 함수
def train(model, train_loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 평가 함수
def evaluate(model, val_loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 학습 루프
best_acc = 0.0
num_epochs = 100

for epoch in range(num_epochs):
    print(f"\n[Epoch {epoch+1}/{num_epochs}]")
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    val_loss, val_acc = evaluate(model, test_loader, criterion)

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")
        print("  [*] Best model saved.")

    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("Early Stopping Triggered!")
        break

print(f"\nBest Accuracy: {best_acc:.4f}")



Loaded pretrained weights for efficientnet-b0

[Epoch 1/100]


Train Loss: 2.5274 | Train Acc: 0.2797 | Val Loss: 10.5117 | Val Acc: 0.2260
  [*] Best model saved.

[Epoch 2/100]


Train Loss: 1.4507 | Train Acc: 0.4718 | Val Loss: 1.6894 | Val Acc: 0.4350
  [*] Best model saved.

[Epoch 3/100]


Train Loss: 1.0423 | Train Acc: 0.7189 | Val Loss: 1.2445 | Val Acc: 0.6667
  [*] Best model saved.

[Epoch 4/100]


Train Loss: 0.8509 | Train Acc: 0.8065 | Val Loss: 0.8584 | Val Acc: 0.8192
  [*] Best model saved.

[Epoch 5/100]


...

Train Loss: 0.5057 | Train Acc: 0.9901 | Val Loss: 0.8436 | Val Acc: 0.8701

[Epoch 32/100]


Train Loss: 0.4981 | Train Acc: 0.9887 | Val Loss: 0.7854 | Val Acc: 0.8701

[Epoch 33/100]


Train Loss: 0.4982 | Train Acc: 0.9915 | Val Loss: 0.7946 | Val Acc: 0.8475

[Epoch 34/100]


Train Loss: 0.4619 | Train Acc: 0.9986 | Val Loss: 0.7675 | Val Acc: 0.8701

[Epoch 35/100]


Train Loss: 0.4617 | Train Acc: 0.9986 | Val Loss: 0.7543 | Val Acc: 0.8814
Early Stopping Triggered!

Best Accuracy: 0.8870


In [2]:
 import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from efficientnet_pytorch import EfficientNet
from tqdm import tqdm

# ✅ SEED 설정 (재현성 확보)
seed = 2021
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ✅ EarlyStopping
class EarlyStopping:
    def __init__(self, patience=10, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False
    def __call__(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# ✅ Residual Block 정의
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if in_channels != out_channels or stride != 1:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = self.shortcut(x)
        x = nn.ReLU()(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        x += residual
        return nn.ReLU()(x)

# ✅ ASPP Block 정의
class ASPP(nn.Module):
    def __init__(self, in_channels, out_channels=256):
        super(ASPP, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.conv3_1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, dilation=1)
        self.conv3_6 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=6, dilation=6)
        self.conv3_12 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=12, dilation=12)
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.conv1_for_global = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.conv1x1_out = nn.Conv2d(out_channels * 5, out_channels, kernel_size=1)

    def forward(self, x):
        conv1_out = self.conv1(x)
        conv3_1_out = self.conv3_1(x)
        conv3_6_out = self.conv3_6(x)
        conv3_12_out = self.conv3_12(x)

        global_avg = self.global_avg_pool(x)
        global_avg = self.conv1_for_global(global_avg)
        global_avg = nn.functional.interpolate(global_avg, size=conv1_out.shape[2:], mode='bilinear', align_corners=True)

        out = torch.cat([conv1_out, conv3_1_out, conv3_6_out, conv3_12_out, global_avg], dim=1)
        return self.conv1x1_out(out)

# ✅ Hybrid Model (EfficientNet + VGG + Residual + ASPP)
class HybridModel(nn.Module):
    def __init__(self, num_classes=6):
        super(HybridModel, self).__init__()
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b0')
        self.efficientnet._fc = nn.Identity()

        self.vgg = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
        self.vgg.classifier = nn.Identity()

        efficientnet_out_features = self.efficientnet._conv_head.out_channels  # 1280
        vgg_out_features = 512  

        self.efficientnet_pool = nn.AdaptiveAvgPool2d((7, 7))
        self.vgg_pool = nn.AdaptiveAvgPool2d((7, 7))

        self.eff_conv1x1 = nn.Conv2d(efficientnet_out_features, 1024, kernel_size=1)
        self.vgg_conv1x1 = nn.Conv2d(vgg_out_features, 1024, kernel_size=1)

        self.res_block = ResidualBlock(2048, 2048)  
        self.aspp = ASPP(in_channels=2048, out_channels=256)

        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))  # GAP 추가
        self.classifier = nn.Sequential(
            nn.Linear(256, 512),  # 256 차원에서 512 차원으로 축소
            nn.ReLU(),
            nn.Dropout(0.3),  
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        eff_features = self.efficientnet.extract_features(x)
        eff_features = self.efficientnet_pool(eff_features)
        eff_features = self.eff_conv1x1(eff_features)

        vgg_features = self.vgg.features(x)
        vgg_features = self.vgg_pool(vgg_features)
        vgg_features = self.vgg_conv1x1(vgg_features)

        fused_features = torch.cat([eff_features, vgg_features], dim=1)  
        fused_features = self.res_block(fused_features)  
        aspp_features = self.aspp(fused_features)
        gap_features = self.global_avg_pool(aspp_features)
        flattened_features = gap_features.view(gap_features.size(0), -1)
        return self.classifier(flattened_features)


# ==============================
# ✅ 데이터 로딩
# ==============================
image_size = 224
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset_path = "C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel"

dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# ==============================
# ✅ 모델, 손실 함수, 최적화기 설정
# ==============================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HybridModel(num_classes=6).to(device)  # 모델을 먼저 생성해야 함!

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# ✅ Early Stopping 적용
early_stopping = EarlyStopping(patience=10)

# ✅ 학습 함수
def train(model, train_loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 평가 함수
def evaluate(model, val_loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 학습 루프
best_acc = 0.0
num_epochs = 100

for epoch in range(num_epochs):
    print(f"\n[Epoch {epoch+1}/{num_epochs}]")
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    val_loss, val_acc = evaluate(model, test_loader, criterion)

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")
        print("  [*] Best model saved.")

    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("Early Stopping Triggered!")
        break

print(f"\nBest Accuracy: {best_acc:.4f}")



Loaded pretrained weights for efficientnet-b0

[Epoch 1/100]


Train Loss: 2.5600 | Train Acc: 0.2627 | Val Loss: 1.6581 | Val Acc: 0.3220
  [*] Best model saved.

[Epoch 2/100]


Train Loss: 1.4925 | Train Acc: 0.4364 | Val Loss: 1.2412 | Val Acc: 0.6158
  [*] Best model saved.

[Epoch 3/100]


Train Loss: 1.0990 | Train Acc: 0.6949 | Val Loss: 1.4725 | Val Acc: 0.5932

[Epoch 4/100]


Train Loss: 0.8299 | Train Acc: 0.8333 | Val Loss: 0.9517 | Val Acc: 0.7853
  [*] Best model saved.

[Epoch 5/100]


...

Train Loss: 0.4939 | Train Acc: 0.9873 | Val Loss: 0.7123 | Val Acc: 0.8870

[Epoch 29/100]


Train Loss: 0.5078 | Train Acc: 0.9788 | Val Loss: 0.7581 | Val Acc: 0.8588

[Epoch 30/100]


Train Loss: 0.5205 | Train Acc: 0.9887 | Val Loss: 0.7982 | Val Acc: 0.8531

[Epoch 31/100]


Train Loss: 0.5028 | Train Acc: 0.9816 | Val Loss: 0.7993 | Val Acc: 0.8588

[Epoch 32/100]


Train Loss: 0.4940 | Train Acc: 0.9873 | Val Loss: 0.7158 | Val Acc: 0.8927
Early Stopping Triggered!

Best Accuracy: 0.9096


In [4]:
 import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from efficientnet_pytorch import EfficientNet
from tqdm import tqdm

# ✅ SEED 설정 (재현성 확보)
seed = 2021
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ✅ EarlyStopping
class EarlyStopping:
    def __init__(self, patience=10, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False
    def __call__(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# ✅ Residual Block 정의
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if in_channels != out_channels or stride != 1:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = self.shortcut(x)
        x = nn.ReLU()(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        x += residual
        return nn.ReLU()(x)

# ✅ ASPP Block 정의
class ASPP(nn.Module):
    def __init__(self, in_channels, out_channels=256):
        super(ASPP, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.conv3_1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, dilation=1)
        self.conv3_6 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=6, dilation=6)
        self.conv3_12 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=12, dilation=12)
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.conv1_for_global = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.conv1x1_out = nn.Conv2d(out_channels * 5, out_channels, kernel_size=1)

    def forward(self, x):
        conv1_out = self.conv1(x)
        conv3_1_out = self.conv3_1(x)
        conv3_6_out = self.conv3_6(x)
        conv3_12_out = self.conv3_12(x)

        global_avg = self.global_avg_pool(x)
        global_avg = self.conv1_for_global(global_avg)
        global_avg = nn.functional.interpolate(global_avg, size=conv1_out.shape[2:], mode='bilinear', align_corners=True)

        out = torch.cat([conv1_out, conv3_1_out, conv3_6_out, conv3_12_out, global_avg], dim=1)
        return self.conv1x1_out(out)

# ✅ Hybrid Model (EfficientNet + VGG + Residual + ASPP)
class HybridModel(nn.Module):
    def __init__(self, num_classes=6):
        super(HybridModel, self).__init__()
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b0')
        self.efficientnet._fc = nn.Identity()

        # ✅ VGG16 → VGG19 변경
        self.vgg = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1)
        self.vgg.classifier = nn.Identity()

        efficientnet_out_features = self.efficientnet._conv_head.out_channels  # 1280
        vgg_out_features = 512  # VGG19의 출력 채널 수

        self.efficientnet_pool = nn.AdaptiveAvgPool2d((7, 7))
        self.vgg_pool = nn.AdaptiveAvgPool2d((7, 7))

        self.eff_conv1x1 = nn.Conv2d(efficientnet_out_features, 1024, kernel_size=1)
        self.vgg_conv1x1 = nn.Conv2d(vgg_out_features, 1024, kernel_size=1)

        self.res_block = ResidualBlock(2048, 2048)  
        self.aspp = ASPP(in_channels=2048, out_channels=256)

        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))  # GAP 추가
        self.classifier = nn.Sequential(
            nn.Linear(256, 512),  # 256 차원에서 512 차원으로 축소
            nn.ReLU(),
            nn.Dropout(0.3),  
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        eff_features = self.efficientnet.extract_features(x)
        eff_features = self.efficientnet_pool(eff_features)
        eff_features = self.eff_conv1x1(eff_features)

        vgg_features = self.vgg.features(x)
        vgg_features = self.vgg_pool(vgg_features)
        vgg_features = self.vgg_conv1x1(vgg_features)

        fused_features = torch.cat([eff_features, vgg_features], dim=1)  
        fused_features = self.res_block(fused_features)  
        aspp_features = self.aspp(fused_features)
        gap_features = self.global_avg_pool(aspp_features)
        flattened_features = gap_features.view(gap_features.size(0), -1)
        return self.classifier(flattened_features)


# ==============================
# ✅ 데이터 로딩
# ==============================
image_size = 224
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset_path = "C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel"

dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# ==============================
# ✅ 모델, 손실 함수, 최적화기 설정
# ==============================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HybridModel(num_classes=6).to(device)  # 모델을 먼저 생성해야 함!

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# ✅ Early Stopping 적용
early_stopping = EarlyStopping(patience=10)

# ✅ 학습 함수
def train(model, train_loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 평가 함수
def evaluate(model, val_loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 학습 루프
best_acc = 0.0
num_epochs = 100

for epoch in range(num_epochs):
    print(f"\n[Epoch {epoch+1}/{num_epochs}]")
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    val_loss, val_acc = evaluate(model, test_loader, criterion)

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")
        print("  [*] Best model saved.")

    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("Early Stopping Triggered!")
        break

print(f"\nBest Accuracy: {best_acc:.4f}")



Loaded pretrained weights for efficientnet-b0


Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to C:\Users\IIALAB/.cache\torch\hub\checkpoints\vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:50<00:00, 11.3MB/s] 



[Epoch 1/100]


Train Loss: 2.7076 | Train Acc: 0.2444 | Val Loss: 1.9248 | Val Acc: 0.2994
  [*] Best model saved.

[Epoch 2/100]


Train Loss: 1.3111 | Train Acc: 0.5946 | Val Loss: 4.3120 | Val Acc: 0.3164
  [*] Best model saved.

[Epoch 3/100]


...

Train Loss: 0.5063 | Train Acc: 0.9774 | Val Loss: 0.8876 | Val Acc: 0.8475

[Epoch 22/100]


Train Loss: 0.5559 | Train Acc: 0.9661 | Val Loss: 0.9375 | Val Acc: 0.7910

[Epoch 23/100]


Train Loss: 0.5181 | Train Acc: 0.9774 | Val Loss: 0.8701 | Val Acc: 0.8305

[Epoch 24/100]


Train Loss: 0.5096 | Train Acc: 0.9845 | Val Loss: 0.7999 | Val Acc: 0.8531

[Epoch 25/100]


Train Loss: 0.4809 | Train Acc: 0.9944 | Val Loss: 0.7787 | Val Acc: 0.8644
Early Stopping Triggered!

Best Accuracy: 0.9040


In [5]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from efficientnet_pytorch import EfficientNet
from tqdm import tqdm

# ✅ SEED 설정 (재현성 확보)
seed = 2021
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ✅ EarlyStopping
class EarlyStopping:
    def __init__(self, patience=10, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False
    def __call__(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# ✅ Residual Block 정의
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if in_channels != out_channels or stride != 1:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = self.shortcut(x)
        x = nn.ReLU()(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        x += residual
        return nn.ReLU()(x)

# ✅ ASPP Block 정의
class ASPP(nn.Module):
    def __init__(self, in_channels, out_channels=256):
        super(ASPP, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.conv3_1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, dilation=1)
        self.conv3_6 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=6, dilation=6)
        self.conv3_12 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=12, dilation=12)
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.conv1_for_global = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.conv1x1_out = nn.Conv2d(out_channels * 5, out_channels, kernel_size=1)

    def forward(self, x):
        conv1_out = self.conv1(x)
        conv3_1_out = self.conv3_1(x)
        conv3_6_out = self.conv3_6(x)
        conv3_12_out = self.conv3_12(x)

        global_avg = self.global_avg_pool(x)
        global_avg = self.conv1_for_global(global_avg)
        global_avg = nn.functional.interpolate(global_avg, size=conv1_out.shape[2:], mode='bilinear', align_corners=True)

        out = torch.cat([conv1_out, conv3_1_out, conv3_6_out, conv3_12_out, global_avg], dim=1)
        return self.conv1x1_out(out)

# ✅ Hybrid Model (EfficientNet + VGG + Residual + ASPP)
class HybridModel(nn.Module):
    def __init__(self, num_classes=6):
        super(HybridModel, self).__init__()
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b0')
        self.efficientnet._fc = nn.Identity()

        # ✅ VGG16 → VGG19 변경
        self.vgg = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1)
        self.vgg.classifier = nn.Identity()

        efficientnet_out_features = self.efficientnet._conv_head.out_channels  # 1280
        vgg_out_features = 512  # VGG19의 출력 채널 수

        self.efficientnet_pool = nn.AdaptiveAvgPool2d((7, 7))
        self.vgg_pool = nn.AdaptiveAvgPool2d((7, 7))

        self.eff_conv1x1 = nn.Conv2d(efficientnet_out_features, 1024, kernel_size=1)
        self.vgg_conv1x1 = nn.Conv2d(vgg_out_features, 1024, kernel_size=1)

        self.res_block = ResidualBlock(2048, 2048)  
        self.aspp = ASPP(in_channels=2048, out_channels=256)

        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))  # GAP 추가
        self.classifier = nn.Sequential(
            nn.Linear(256, 512),  # 256 차원에서 512 차원으로 축소
            nn.ReLU(),
            nn.Dropout(0.3),  
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        eff_features = self.efficientnet.extract_features(x)
        eff_features = self.efficientnet_pool(eff_features)
        eff_features = self.eff_conv1x1(eff_features)

        vgg_features = self.vgg.features(x)
        vgg_features = self.vgg_pool(vgg_features)
        vgg_features = self.vgg_conv1x1(vgg_features)
        vgg_features = nn.Dropout(0.3)(vgg_features)


        fused_features = torch.cat([eff_features, vgg_features], dim=1)  
        fused_features = self.res_block(fused_features)  
        aspp_features = self.aspp(fused_features)
        gap_features = self.global_avg_pool(aspp_features)
        flattened_features = gap_features.view(gap_features.size(0), -1)
        return self.classifier(flattened_features)


# ==============================
# ✅ 데이터 로딩
# ==============================
image_size = 224
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset_path = "C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel"

dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# ==============================
# ✅ 모델, 손실 함수, 최적화기 설정
# ==============================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HybridModel(num_classes=6).to(device)  # 모델을 먼저 생성해야 함!

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# ✅ Early Stopping 적용
early_stopping = EarlyStopping(patience=10)

# ✅ 학습 함수
def train(model, train_loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 평가 함수
def evaluate(model, val_loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 학습 루프
best_acc = 0.0
num_epochs = 100

for epoch in range(num_epochs):
    print(f"\n[Epoch {epoch+1}/{num_epochs}]")
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    val_loss, val_acc = evaluate(model, test_loader, criterion)

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")
        print("  [*] Best model saved.")

    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("Early Stopping Triggered!")
        break

print(f"\nBest Accuracy: {best_acc:.4f}")



Loaded pretrained weights for efficientnet-b0

[Epoch 1/100]


Train Loss: 2.9099 | Train Acc: 0.2641 | Val Loss: 2.4677 | Val Acc: 0.3842
  [*] Best model saved.

[Epoch 2/100]


Train Loss: 1.3733 | Train Acc: 0.5706 | Val Loss: 1.2310 | Val Acc: 0.6328
  [*] Best model saved.

[Epoch 3/100]


Train Loss: 1.0689 | Train Acc: 0.6977 | Val Loss: 1.5944 | Val Acc: 0.6441
  [*] Best model saved.

[Epoch 4/100]


Train Loss: 0.8537 | Train Acc: 0.8404 | Val Loss: 1.2747 | Val Acc: 0.6384

[Epoch 5/100]


...

Train Loss: 0.5247 | Train Acc: 0.9718 | Val Loss: 0.7839 | Val Acc: 0.8814

[Epoch 16/100]


Train Loss: 0.5124 | Train Acc: 0.9845 | Val Loss: 0.8392 | Val Acc: 0.8814

[Epoch 17/100]


Train Loss: 0.5185 | Train Acc: 0.9873 | Val Loss: 0.7866 | Val Acc: 0.8757

[Epoch 18/100]


Train Loss: 0.4787 | Train Acc: 0.9929 | Val Loss: 0.7732 | Val Acc: 0.8757

[Epoch 19/100]


Train Loss: 0.4674 | Train Acc: 0.9887 | Val Loss: 0.7638 | Val Acc: 0.8644
Early Stopping Triggered!

Best Accuracy: 0.8927
